In [1]:
import torch
import torch.nn as nn

In [2]:
class SimpleModel(nn.Module):

    def __init__(self, in_ch, out_ch):

        super().__init__()
        
        self.in_ch = in_ch
        self.out_ch = out_ch
        
    def forward(self, x):

        return nn.Sequential(
            nn.Linear(self.in_ch, 32),
            nn.ReLU(),
            nn.Linear(32, self.out_ch, bias = False),
            nn.ReLU()
        )(x)

# Задание 14

## 1.

In [3]:
img = [1, 2, 1, 2, 4, 2, 1, 2, 1, -1, 0, 1, 0, 0, 0, 1, 0, -1, 0, 1, 0, 1, 2, 1, 0, 1, 0]
img = torch.tensor(img, dtype = torch.float).reshape(3, 3, 3)

img1 = [1, 0, 0, 1, 0, -1, -1, 0, 1, -1, 0, 0]
img1 = torch.tensor(img1, dtype = torch.float).reshape(3, 2, 2)

In [4]:
conv = torch.nn.Conv2d(3, 3, kernel_size = 2, padding = 1, stride = 1, dilation = 2, bias = False)

print(conv(img), '\n')
print(conv(img1))

tensor([[[-3.6121e-01,  1.9278e-01,  7.4678e-01],
         [ 1.6323e-01,  2.4235e-01,  4.3452e-04],
         [ 6.8768e-01, -2.9116e-02, -7.4591e-01]],

        [[-1.3368e+00, -8.7429e-01, -4.1177e-01],
         [-1.4875e+00, -7.3474e-01,  1.1528e-02],
         [-1.6382e+00, -6.0167e-01,  4.3483e-01]],

        [[-1.1343e+00, -9.7110e-01, -8.0787e-01],
         [-3.4850e-01, -4.9166e-01, -1.4988e-01],
         [ 4.3732e-01,  4.7271e-01,  5.0811e-01]]], grad_fn=<SqueezeBackward1>) 

tensor([[[-0.0524, -0.1676],
         [-0.1687, -0.1032]],

        [[-0.2046, -0.1020],
         [ 0.4655,  0.1433]],

        [[-0.2761,  0.1537],
         [ 0.0610,  0.0037]]], grad_fn=<SqueezeBackward1>)


## 2.

In [5]:
def get_receptive_field(layers_cnt, kernels_size, strides, paddings, dilations):

    res, s = 1, 1

    for l in range(layers_cnt):
        for i in range(l):

            s *= strides[i]
            res += dilations[l] * (kernels_size[l] - 1) * s 

    start_id, end_id = 0, 0

    for l in range(layers_cnt -1, -1, -1):

        start_id = -paddings[l] + start_id * strides[l]
        end_id = -paddings[l] + end_id * strides[l] + kernels_size[l] - 1

    return res, start_id, end_id

In [6]:
kernels_size = [7, 3, 2, 3, 3]
strides = [1, 2, 2, 1, 3]
paddings = [3, 1, 0, 100500, 28]
dilations = [1, 1, 1, 2, 3]

print(get_receptive_field(5, kernels_size, strides, paddings, dilations))

(590, -402116, -402090)


## 3.

In [7]:
def new_size(input_size, kernel_size, padding, stride, dilation):

    return int(1 + (input_size + 2 * padding - dilation * (kernel_size - 1) - 1) / stride)

In [8]:
size = new_size(224, kernel_size = 7, padding = 3, stride = 2, dilation = 1)
size = new_size(size, kernel_size = 3, padding = 1, stride = 1, dilation = 1)
size = size / 2
size = new_size(size, kernel_size = 3, padding = 1, stride = 2, dilation = 3)

print('Output image size:', end = ' ')
print('(', size, ',', size, ')')

Output image size: ( 26 , 26 )
